## Modelagem

Nesta série de scripts iremos começar o processo de modelagem. O método que mais se aplica ao nosso conjunto de dados é a análise de dados em painel. Aqui iremos aplicar este primeiro método e depois iremos explorar outras opções. Usei o exemplo desse [link](https://bashtage.github.io/linearmodels/panel/examples/examples.html).

In [1]:
# Carregar librarias
import pandas as pd
import statsmodels.api as sm
from linearmodels.datasets import wage_panel
from linearmodels.panel import PooledOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import BetweenOLS
from linearmodels.panel import compare
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Carregar dataset
data_sugarcane = pd.read_csv("new_data_all.csv", sep =  ',').drop(['Unnamed: 0'], axis = 1)

In [7]:
# Visão dos dados
data_sugarcane.head()

,CD_MUN,MUN,Year,Yield,AP,AC,QP,VP,AET,NDVI,NPP,SOIL,TMIN,TMMX,PR
0,3500105,Adamantina (SP),2000,50000.0,7000.0,7000.0,350000.0,6244.0,704,5174,4047,166,163,288,91
1,3500204,Adolfo (SP),2000,NaN,NaN,NaN,NaN,NaN,716,4080,5245,255,162,290,94
2,3500303,Aguaí (SP),2000,80000.0,5250.0,5250.0,420000.0,7493.0,675,5614,8672,399,151,272,109
3,3500550,Águas de Santa Bárbara (SP),2000,NaN,NaN,NaN,NaN,NaN,776,5388,8146,246,142,270,95
4,3500709,Agudos (SP),2000,70000.0,8000.0,8000.0,560000.0,9990.0,746,5969,9080,248,146,274,94


In [8]:
# Converter o ano para categórico
year_sugar = pd.Categorical(data_sugarcane.Year)
year_sugar

[2000, 2000, 2000, 2000, 2000, ..., 2020, 2020, 2020, 2020, 2020]
Length: 10185
Categories (21, int64): [2000, 2001, 2002, 2003, ..., 2017, 2018, 2019, 2020]

In [9]:
# Criar o index utilizando o ano e o código do municipio
data_sugar = data_sugarcane.set_index(['CD_MUN', 'Year'])

In [10]:
# Adicionar a coluna do ano
data_sugar['Year'] = year_sugar

In [11]:
# Info
data_sugar.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10185 entries, (3500105, 2000) to (3557303, 2020)
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   MUN     10185 non-null  object  
 1   Yield   9525 non-null   float64 
 2   AP      9536 non-null   float64 
 3   AC      9527 non-null   float64 
 4   QP      9527 non-null   float64 
 5   VP      9527 non-null   float64 
 6   AET     10185 non-null  int64   
 7   NDVI    10185 non-null  int64   
 8   NPP     10185 non-null  int64   
 9   SOIL    10185 non-null  int64   
 10  TMIN    10185 non-null  int64   
 11  TMMX    10185 non-null  int64   
 12  PR      10185 non-null  int64   
 13  Year    10185 non-null  category
dtypes: category(1), float64(5), int64(7), object(1)
memory usage: 1.1+ MB


In [14]:
# Selecionar as variáveis que serão utilizadas na modelagem
exog_vars_sugar = ['AP', 'AC', 'QP', 'VP', 'AET', 'NDVI', 'NPP', 'SOIL', 'TMIN', 'TMMX', 'PR','Year']

In [15]:
# Adicionar a constante
exog_sugar = sm.add_constant(data_sugar[exog_vars_sugar])
exog_sugar.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10185 entries, (3500105, 2000) to (3557303, 2020)
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   const   10185 non-null  float64 
 1   AP      9536 non-null   float64 
 2   AC      9527 non-null   float64 
 3   QP      9527 non-null   float64 
 4   VP      9527 non-null   float64 
 5   AET     10185 non-null  int64   
 6   NDVI    10185 non-null  int64   
 7   NPP     10185 non-null  int64   
 8   SOIL    10185 non-null  int64   
 9   TMIN    10185 non-null  int64   
 10  TMMX    10185 non-null  int64   
 11  PR      10185 non-null  int64   
 12  Year    10185 non-null  category
dtypes: category(1), float64(5), int64(7)
memory usage: 1016.1 KB


In [17]:
# Criar e rodar o modelo - Simple OLS
mod_sugar = PooledOLS(data_sugar.Yield, exog_sugar)
pooled_res_sugar = mod_sugar.fit()
print(pooled_res_sugar)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  Yield   R-squared:                        0.3661
Estimator:                  PooledOLS   R-squared (Between):              0.4224
No. Observations:                9525   R-squared (Within):               0.3377
Date:                Wed, Jul 27 2022   R-squared (Overall):              0.3661
Time:                        07:59:57   Log-likelihood                -1.016e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      176.83
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                 F(31,9493)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             176.83
                            

In [18]:
# Criar e rodar o modelo  - Random effects
mod_random_sugar = RandomEffects(data_sugar.Yield, exog_sugar)
re_res_sugar = mod_random_sugar.fit()
print(re_res_sugar)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  Yield   R-squared:                        0.3583
Estimator:              RandomEffects   R-squared (Between):              0.4002
No. Observations:                9525   R-squared (Within):               0.3420
Date:                Wed, Jul 27 2022   R-squared (Overall):              0.3619
Time:                        08:01:28   Log-likelihood                -1.002e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      171.02
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                 F(31,9493)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             161.71
                            

In [19]:
# Decomponer a variancia
re_res_sugar.variance_decomposition

Effects                   2.688133e+07
Residual                  8.041024e+07
Percent due to Effects    2.505446e-01
Name: Variance Decomposition, dtype: float64

Rodar o modelo sem considerar o ano (Between OLS)

In [20]:
# Excluir o ano das variavéis
exog_vars_sugar_sem_year = ['AP', 'AC', 'QP', 'VP', 'AET', 'NDVI', 'NPP', 'SOIL', 'TMIN', 'TMMX', 'PR']

In [21]:
# Adicionar a constante
exog_sem_year = sm.add_constant(data_sugar[exog_vars_sugar_sem_year])

In [22]:
# Criar e rodar o modelo sem o efeito do ano
mod_sem_year = BetweenOLS(data_sugar.Yield, exog_sem_year)
be_res_sugar = mod_sem_year.fit(reweight = True)
print(be_res_sugar)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                  Yield   R-squared:                        0.4360
Estimator:                 BetweenOLS   R-squared (Between):              0.4309
No. Observations:                 485   R-squared (Within):               0.1402
Date:                Wed, Jul 27 2022   R-squared (Overall):              0.2334
Time:                        08:02:17   Log-likelihood                   -4863.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      69.221
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                  F(11,473)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             33.238
                            

## Comparar os modelos

In [23]:
print(compare({"BE": be_res_sugar,'RE': re_res_sugar, "Pooled": pooled_res_sugar}))

                        Model Comparison                       
                                BE              RE       Pooled
---------------------------------------------------------------
Dep. Variable                Yield           Yield        Yield
Estimator               BetweenOLS   RandomEffects    PooledOLS
No. Observations               485            9525         9525
Cov. Est.               Unadjusted      Unadjusted   Unadjusted
R-squared                   0.4360          0.3583       0.3661
R-Squared (Within)          0.1402          0.3420       0.3377
R-Squared (Between)         0.4309          0.4002       0.4224
R-Squared (Overall)         0.2334          0.3619       0.3661
F-statistic                 69.221          171.02       176.83
P-value (F-stat)            0.0000          0.0000       0.0000
===================== ============ =============== ============
const                    1.608e+05       9.353e+04    1.278e+05
                          (9.3964)      